# Mini Project: Proses ETL Cabang Perusahaan Retail
## Data Manipulation with Pandas - Part 1

Mini Project ini merupakan tugas akhir dari materi Data Manipulation with Pandas - Part 1 dalam Data Analyst Career Track (Python) DQLab. Library `pandas` digunakan sebagai <i>tools</i> utama dalam tugas ini. Sebagai pengantar, ETL---<i>yang berarti Extract, Transform, dan Load</i>---merupakan proses integrasi data dari beberapa sumber data menjadi satu <i>data store</i> yang konsisten dan dikumpulkan dalam <i>data warehouse</i> atau bentuk sistem lainnya.

ETL umum digunakan oleh organisasi untuk:
1. Ekstraksi data dari sistem terdahulu
2. Pembersihan data untuk memperbaiki kualitas dan konsistensi data
3. Pemuatan data ke dalam database

Sumber: <a href="https://www.ibm.com/cloud/learn/etl">IBM Cloud Learn Hub</a>

Dalam mini project ini, proses ETL berfokus pada tahapan <b>Transform</b>. Berikut ini adalah langkah-langkah per <i>script-block</i> yang saya sediakan:

## Import Library Pandas

In [1]:
# Import library yang dibutuhkan
import pandas as pd

## Inspeksi Data (1): Baca Dataframe

In [2]:
# Baca dataset
df = pd.read_csv("https://storage.googleapis.com/dqlab-dataset/retail_raw_test.csv", low_memory=False)
df

,order_id,order_date,customer_id,city,province,brand,quantity,item_price,product_value
0,1730350,"Dec 11, 2019",'13447,Surakarta,Jawa Tengah,BRAND_F,'24,'113000,1374.0
1,1677490,"Jul 31, 2019",'0,NaN,NaN,BRAND_F,'1,'1164000,1370.0
2,1704211,"Oct 18, 2019",'16128,Jakarta Pusat,DKI Jakarta,BRAND_H,'12,'747000,1679.0
3,1679695,"Aug 07, 2019",'16225,Yogyakarta,Yogyakarta,BRAND_H,'6,'590000,1708.0
4,1679080,"Aug 05, 2019",'0,NaN,NaN,BRAND_E,'2,'740000,1201.0
...,...,...,...,...,...,...,...,...,...
4995,1616509,"Jan 08, 2019",'12748,Jakarta Utara,DKI Jakarta,BRAND_B,'1,'1325000,449.0
4996,1724851,"Dec 03, 2019",'0,NaN,NaN,BRAND_H,'2,'1458000,1685.0
4997,1715698,"Nov 13, 2019",'16885,Bekasi,Jawa Barat,BRAND_S,'1,'450000,3206.0
4998,1668061,"Jul 03, 2019",'13571,Jakarta Utara,DKI Jakarta,BRAND_W,'25,'159000,4126.0


Dataframe memiliki <b>5000 baris</b> dan <b>9 kolom</b>.

Sekilas, ada beberapa masalah yang tampak jelas pada dataframe, yaitu:<br/>
(1) kolom `customer_id`, `quantity`, dan `item_price` memiliki tanda apostrof</br>
(2) kolom `city` dan `province` memiliki nilai `NaN`/null.

<u><b>Solusi</b></u><br/>
Masalah (1) : gunakan <i>string splitting</i> disertai dengan penggantian tipe data `string` menjadi `int64`<br/>
Masalah (2) : isi data `NaN` dengan `unknown`

## Inspeksi Data (2): Informasi Dataframe

In [3]:
# Tampilkan informasi dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       5000 non-null   int64  
 1   order_date     5000 non-null   object 
 2   customer_id    5000 non-null   object 
 3   city           3802 non-null   object 
 4   province       3802 non-null   object 
 5   brand          4995 non-null   object 
 6   quantity       5000 non-null   object 
 7   item_price     5000 non-null   object 
 8   product_value  4995 non-null   float64
dtypes: float64(1), int64(1), object(7)
memory usage: 351.7+ KB


Method `info()` pada dataframe dapat menampilkan beberapa informasi penting seperti <i>non-null count</i> dan tipe data (<i>dtype</i>). Selain kolom `city` dan `province`, kolom lainnya yang masih memiliki <i>missing values</i> adalah kolom `brand` dan `product_value`. Sebagai tambahan, tipe data kolom `order_date` dapat diubah menjadi `datetime64` dengan melakukan transformasi pada kolom tersebut terlebih dahulu.

Solusi:
1. <i>Missing values</i> pada kolom `brand` dapat diisi dengan `no_brand`, sedangkan <i>missing values</i> pada kolom `product_value` akan dibahas di bagian khusus
2. Data tanggal pada `order_date` diubah formatnya menjadi bentuk `YYYY-MM-DD` dengan menggunakan <i>customized function</i>.

## Koreksi Value dan Tipe Data Customer ID, Quantity dan Item Price

In [4]:
# Perbaiki data sekaligus ubah tipe data customer_id, quantity, dan item_price
df["customer_id"] = df["customer_id"].apply(lambda x: x.split("'")[1]).astype("int64")
df["quantity"] = df["quantity"].apply(lambda x: x.split("'")[1]).astype("int64")
df["item_price"] = df["item_price"].apply(lambda x: x.split("'")[1]).astype("int64")
df.dtypes

order_id           int64
order_date        object
customer_id        int64
city              object
province          object
brand             object
quantity           int64
item_price         int64
product_value    float64
dtype: object

<img src="/images/split-method.png" style="margin-left:auto;margin-right:auto;width:40%">

Koreksi <i>value</i> pada kolom `customer_id`, `quantity` dan `item_price` dilakukan dengan mengaplikasikan (<i>series method</i> `apply()`) <i>string method</i> `split()` yang dikemas dalam fungsi `lambda`. Method `split()` akan memecah <i>string object</i> menjadi dua bagian dengan separatornya adalah tanda apostrof ('). 

Tipe data dari ketiga kolom tersebut diubah dengan <i>series method</i> `astype()`.

## Transformasi Product Value

In [5]:
# Transform "product_value" supaya bentuknya seragam dengan format "PXXXX", assign ke kolom baru "product_id",
# dan drop kolom "product_value", jika terdapat nan gantilah dengan "unknown"
# Buat fungsi impute_product_value()
import math
def impute_product_value(val):
	if math.isnan(val):
		return "unknown"
	else:
		return 'P' + '{:0>4}'.format(str(val).split('.')[0])

# Buat kolom "product_id"
df["product_id"] = df["product_value"].apply(lambda x: impute_product_value(x))
# Hapus kolom "product_value"
df.drop(["product_value"], axis=1, inplace=True)
# Cetak 5 data teratas
df.head()

,order_id,order_date,customer_id,city,province,brand,quantity,item_price,product_id
0,1730350,"Dec 11, 2019",13447,Surakarta,Jawa Tengah,BRAND_F,24,113000,P1374
1,1677490,"Jul 31, 2019",0,NaN,NaN,BRAND_F,1,1164000,P1370
2,1704211,"Oct 18, 2019",16128,Jakarta Pusat,DKI Jakarta,BRAND_H,12,747000,P1679
3,1679695,"Aug 07, 2019",16225,Yogyakarta,Yogyakarta,BRAND_H,6,590000,P1708
4,1679080,"Aug 05, 2019",0,NaN,NaN,BRAND_E,2,740000,P1201


Kolom `product_value` akan diubah menjadi kolom `product_id` dengan format `PXXXX`. Namun, kolom `product_value` masih memiliki <i>missing values</i> seperti yang telah dipaparkan sebelumnya.

Fungsi `impute_product_value()` dirancang untuk mentransformasi kolom `product_value` yang masih memiliki <i>missing values</i>. Data `NaN` akan diubah menjadi `"unknown"`, sedangkan data numerik akan diubah menjadi kode produk dengan format `PXXXX`. 

Kode produk ini dibuat dengan <i>string formatting</i> dengan prosedur:
1. Ubah data numerik menjadi tipe data <i>string</i> (`str()`).
2. Ambil kumpulan angka di depan tanda dot desimal (.) dengan <i>string method</i> `split()`.
3. Format <i>string</i> angka tersebut dengan menambahkan <i>padding substitution</i> berupa karakter "0". Hal ini berfungsi untuk mengantisipasi adanya data numerik yang angkanya kurang dari 4 karakter.
4. Gabungkan <i>string</i> angka dengan huruf "P" untuk membentuk kode produk `PXXXX`.


## Transformasi Order Date

In [6]:
# Tranform order_date menjadi value dengan format "YYYY-mm-dd"
months_dict = {
   "Jan":"01",
   "Feb":"02",
   "Mar":"03",
   "Apr":"04",
   "May":"05",
   "Jun":"06",
   "Jul":"07",
   "Aug":"08",
   "Sep":"09",
   "Oct":"10",
   "Nov":"11",
   "Dec":"12"
}
df["order_date"] = pd.to_datetime(
    df["order_date"].apply(lambda x: str(x)[-4:] + "-" + months_dict[str(x)[:3]] + "-" + str(x)[4:7]))
df.dtypes

order_id                int64
order_date     datetime64[ns]
customer_id             int64
city                   object
province               object
brand                  object
quantity                int64
item_price              int64
product_id             object
dtype: object

<img src="/images/string-slicing.png" style="margin-left:auto;margin-right:auto;width:70%">

Transformasi `order_date` dilakukan dengan mengambil bagian dari <i>string</i> data tanggal dengan melakukan <i>string slicing</i> berdasarkan indeksnya.

Khusus untuk bagian "bulan", bagian <i>string</i>-nya dimasukkan ke dalam <i>dictionary</i> `months_dict` yang berisikan <i>key</i> berupa nama bulan dalam 3 huruf (contoh: "Dec") dan <i>value</i> berupa bulan dalam bentuk <i>string</i> angka (contoh: "12").`

## Penanganan Missing Values pada Dataframe

In [7]:
# Mengatasi data yang hilang di beberapa kolom
# Kolom "city" dan "province" masih memiliki missing value,
# nilai yang hilang di kedua kolom ini diisi saja dengan "unknown"
df[["city","province"]] = df[["city","province"]].fillna("unknown")
# Kolom brand juga masih memiliki missing value, Ganti value NaN menjadi "no_brand"
df["brand"] = df["brand"].fillna("no_brand")
# Cek apakah masih terdapat missing value di seluruh kolom 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     5000 non-null   int64         
 1   order_date   5000 non-null   datetime64[ns]
 2   customer_id  5000 non-null   int64         
 3   city         5000 non-null   object        
 4   province     5000 non-null   object        
 5   brand        5000 non-null   object        
 6   quantity     5000 non-null   int64         
 7   item_price   5000 non-null   int64         
 8   product_id   5000 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 351.7+ KB


<i>Missing values</i> atau data `NaN` pada kolom kategorikal dapat diisi dengan <i>string</i> `"unknown"` untuk kolom `city` dan `province` serta `"no_brand"` untuk kolom `brand`. <i>Series method</i> `fillna()` digunakan untuk mengisi data `NaN` tersebut.

## Manipulasi Dataframe: Feature Engineering (1)

In [8]:
# Membuat kolom baru "city/province" dengan menggabungkan kolom "city"
# dan kolom "province" dan delete kolom asalnya
df["city/province"] = df["city"] + "/" + df["province"]
# drop kolom "city" dan "province" karena telah digabungkan
df.drop(["city","province"], axis=1, inplace=True)
# Cetak 5 data teratas
df.head()

,order_id,order_date,customer_id,brand,quantity,item_price,product_id,city/province
0,1730350,2019-12-11,13447,BRAND_F,24,113000,P1374,Surakarta/Jawa Tengah
1,1677490,2019-07-31,0,BRAND_F,1,1164000,P1370,unknown/unknown
2,1704211,2019-10-18,16128,BRAND_H,12,747000,P1679,Jakarta Pusat/DKI Jakarta
3,1679695,2019-08-07,16225,BRAND_H,6,590000,P1708,Yogyakarta/Yogyakarta
4,1679080,2019-08-05,0,BRAND_E,2,740000,P1201,unknown/unknown


<i>Feature engineering</i> dapat dilakukan dengan memanfaatkan kolom yang sudah ada. Dalam hal ini, kolom `city` dan `province` disatukan menjadi satu kolom `city/province`. Hal ini dapat dilakukan karena baik `city` dan `province` sama-sama mewakili entitas suatu wilayah yang sama.

Proses penyatuan sama seperti <i>string concatenation</i>, hanya perlu menambahkan masing-masing komponen dengan urutan tertentu, dan bisa ditambahkan karakter tertentu seperti "/" pada  data `city/province` yang dibuat di samping.

## Manipulasi Dataframe: Multiindexing

In [9]:
# Membuat hierarchical index yang terdiri dari kolom "city/province", "order_date",
# "customer_id", "order_id", "product_id"
df = df.set_index(["city/province","order_date","customer_id","order_id","product_id"])
# urutkanlah berdasarkan index yang baru
df = df.sort_index()
# Cetak 5 data teratas
df.head()


brand  \
city/province          order_date customer_id order_id product_id            
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936       BRAND_K   
                       2019-11-12 12360       1715116  P0758       BRAND_C   
                                                       P3042       BRAND_R   
                       2019-12-09 12374       1729036  P1660       BRAND_G   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628       BRAND_C   

                                                                   quantity  \
city/province          order_date customer_id order_id product_id             
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936             24   
                       2019-11-12 12360       1715116  P0758              8   
                                                       P3042             12   
                       2019-12-09 12374       1729036  P1660              4   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628             12   

                                                                   item_price  
city/province          order_date customer_id order_id product_id              
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936           450000  
                       2019-11-12 12360       1715116  P0758           695000  
                                                       P3042           310000  
                       2019-12-09 12374       1729036  P1660          2795000  
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           695000

Kolom dapat digunakan sebagai indeks dengan <i>dataframe method</i> `set_index()`. Urutan nama kolom pada <i>argument</i> `set_index()` mengartikan urutan multi-indeks yang akan dihasilkan. Dalam kasus ini, kolom `city/province` menjadi indeks yang paling pertama/terluar pada tabel di samping. Umumnya, kolom yang digunakan di awal haruslah yang memiliki sedikit kategori atau tingkat agregasinya lebih besar.

Masih ingat dengan <i>padding substitution</i> pada bagian sebelumnya? Contoh kode `P0628` ini berasal dari `product_value` `628.00`. <i>Padding substitution</i> berfungsi untuk mengisi angka "0" pada kode ini.

Baca lebih lanjut di: https://www.geeksforgeeks.org/python-string-format-method/

## Manipulasi Dataframe: Feature Engineering (2)

In [10]:
# Membuat kolom "total_price" yang formula nya perkalian antara kolom "quantity" dan kolom "item_price"
df["total_price"] = df["quantity"] * df["item_price"]
# Cetak 5 data teratas
df.head()

brand  \
city/province          order_date customer_id order_id product_id            
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936       BRAND_K   
                       2019-11-12 12360       1715116  P0758       BRAND_C   
                                                       P3042       BRAND_R   
                       2019-12-09 12374       1729036  P1660       BRAND_G   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628       BRAND_C   

                                                                   quantity  \
city/province          order_date customer_id order_id product_id             
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936             24   
                       2019-11-12 12360       1715116  P0758              8   
                                                       P3042             12   
                       2019-12-09 12374       1729036  P1660              4   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628             12   

                                                                   item_price  \
city/province          order_date customer_id order_id product_id               
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936           450000   
                       2019-11-12 12360       1715116  P0758           695000   
                                                       P3042           310000   
                       2019-12-09 12374       1729036  P1660          2795000   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           695000   

                                                                   total_price  
city/province          order_date customer_id order_id product_id               
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936          10800000  
                       2019-11-12 12360       1715116  P0758           5560000  
                                                       P3042           3720000  
                       2019-12-09 12374       1729036  P1660          11180000  
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           8340000

Selain menggabungkan data, <i>feature engineering</i> juga dapat dilakukan dengan manipulasi data seperti perkalian data kolom `quantity` dengan `item_price` untuk menghasilkan kolom `total_price`.

## Manipulasi Dataframe: Slicing

In [11]:
# Slice dataset agar hanya terdapat data bulan Januari 2019
idx = pd.IndexSlice
df_jan2019 = df.loc[idx[:, "2019-01-01":"2019-01-31"], :]
df_jan2019.head()

brand  \
city/province          order_date customer_id order_id product_id            
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628       BRAND_C   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597       BRAND_G   
                       2019-01-10 17392       1617952  P2137       BRAND_M   
                       2019-01-14 15527       1618828  P3115       BRAND_S   
                       2019-01-29 13253       1620289  P0099       BRAND_A   

                                                                   quantity  \
city/province          order_date customer_id order_id product_id             
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628             12   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597              9   
                       2019-01-10 17392       1617952  P2137              2   
                       2019-01-14 15527       1618828  P3115              1   
                       2019-01-29 13253       1620289  P0099             12   

                                                                   item_price  \
city/province          order_date customer_id order_id product_id               
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           695000   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597           520000   
                       2019-01-10 17392       1617952  P2137          1062000   
                       2019-01-14 15527       1618828  P3115          1045000   
                       2019-01-29 13253       1620289  P0099           450000   

                                                                   total_price  
city/province          order_date customer_id order_id product_id               
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           8340000  
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597           4680000  
                       2019-01-10 17392       1617952  P2137           2124000  
                       2019-01-14 15527       1618828  P3115           1045000  
                       2019-01-29 13253       1620289  P0099           5400000

Slicing dataframe yang memiliki multi-indeks lebih mudah dilakukan dengan bantuan pd.IndexSlice (yang disimpan dalam variabel idx). Pada gambar di samping

`idx[:, "2019-01-01":"2019-01-31"]`

ekivalen dengan

`(slice(None, None, None), slice('2019-01-01', '2019-01-31', None))`

yang berfungsi untuk slicing baris yang indeks order_date-nya mulai 2019-01-01 hingga 2019-01-31.

Baca lebih lanjut di:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.IndexSlice.html
https://www.w3schools.com/python/ref_func_slice.asp

<p style="text-align:center;font-weight:bold">********** END OF PROJECT **********</p>